# Pilot Scoring Code 🛬🐍

In [92]:
import pandas as pd
import numpy as np
import os
import csv
import math
import time
import traceback
import datetime
from google.colab import drive
import warnings

warnings.filterwarnings('ignore')
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
SCORE_OUTPUTS_DIR = '/content/drive/MyDrive/Missed Approach Scoring/Score Outputs'
if not os.path.exists(SCORE_OUTPUTS_DIR):
    os.mkdir(SCORE_OUTPUTS_DIR)
    print(f"Directory '{SCORE_OUTPUTS_DIR}' created successfully.")
else:
    print(f"Directory '{SCORE_OUTPUTS_DIR}' already exists.")


SUMMARY_SCORE_OUTPUTS_DIR = os.path.join(SCORE_OUTPUTS_DIR, 'Summary')
if not os.path.exists(SUMMARY_SCORE_OUTPUTS_DIR):
    os.mkdir(SUMMARY_SCORE_OUTPUTS_DIR)
    print(f"Directory '{SUMMARY_SCORE_OUTPUTS_DIR}' created successfully.")
else:
    print(f"Directory '{SUMMARY_SCORE_OUTPUTS_DIR}' already exists.")

Directory '/content/drive/MyDrive/Missed Approach Scoring/Score Outputs' already exists.
Directory '/content/drive/MyDrive/Missed Approach Scoring/Score Outputs/Summary' already exists.


In [94]:
# --- Dataref Column Names ---
TIME_COL = 'missn,_time' # Total time since the flight got reset by something
ALT_MSL_COL = 'p-alt,ftMSL' # Indicated height (altitude), MSL, in feet, primary system, based on pilots barometric pressure input
IAS_COL = '_Vind,_kias' # Indicated airspeed in knots, pilot
HEADING_COL = 'hding,__mag' # Indicated magnetic heading, in degrees. Source: electric gyro. Side: Pilot
VVI_COL = '__VVI,__fpm' # Indicated vertical speed in feet per minute, pilot system
TURN_RATE_COL = 'turnrate,__deg' # Indicated rate-of-turn, in degrees deflection, for newer roll-augmented turn-indicators. Pilot side.
ROLL_COL = '_roll,__deg' # Indicated roll, in degrees, positive right. Source: electric gyro. Side: Pilot
NAV1_HDEF_COL = 'pilN1,h-def' # CDI lateral deflection in dots, nav1, pilot
NAV1_VDEF_COL = 'pilN1,v-def' # CDI vertical deflection in dots, nav1, pilot
NAV1_DME_COL = 'pilN1,dme-d' # Our distance in nautical miles from the beacon tuned in on nav1. override_navneedles
NAV1_FLAG_COL = 'pi1N1,flag' # Nav-To-From indication, nav1, pilot, 0 is flag, 1 is to, 2 is from
IS_PAUSED = 'is_paused' # Whether X-plane was paused (0 for not paused, 1 for paused)

VOR_TRACKING_COLS = [TIME_COL, ALT_MSL_COL, IAS_COL, NAV1_HDEF_COL, IS_PAUSED]
MISSED_APPROACH_COLS = [TIME_COL, ALT_MSL_COL, IAS_COL, HEADING_COL, VVI_COL, TURN_RATE_COL, ROLL_COL, NAV1_HDEF_COL, NAV1_VDEF_COL, NAV1_DME_COL, NAV1_FLAG_COL, IS_PAUSED]

# Helper Functions

## Segment Detection for VOR and Missed Approach

VOR

In [95]:

START_DIST_SEAL_BEACH = 18
END_DIST_SEAL_BEACH = 3

def detect_vor_tracking_segment(df):
    df = df.copy()
    df['segment'] = "Undetermined"

    for i, row in df.iterrows():
        if row[NAV1_DME_COL] <= START_DIST_SEAL_BEACH and row[NAV1_DME_COL] >= END_DIST_SEAL_BEACH:
            df.at[i, 'segment'] = "VOR_Tracking" # Task begins at 18 nm from Seal Beach. Ends at 3 nm

    df = df[df['segment'] != "Undetermined"]
    df = df.reset_index(drop=True)

    return df

Missed Approach

In [96]:
# --- Constants and Definitions for detecting Missed Approach segments/sections ---
FINAL_APP_START_DME = 6.9 # Start criteria for segment 1. Begins at Glideslope Intercept at HILLZ (6.9 DME from Runway)​
FINAL_APP_END_ALT_MSL = 900 # End criteria for segment 1. Final approach: Descend to decision altitude (Ends at 900 MSL)
TRANSITION_END_ALT_MSL = 1200 # End criteria for segment 2. Transition to missed approach (Begins @900 ft MSL -> 1200 ft MSL)
CLIMB_INTERCEPT_END_ALT_MSL = 3100 # End criteria for segment 3. Climb to intercept segment (Ending at 3100 MSL)
NAV1_HORIZONTAL_DEF_MIN = 0.2 # End criteria for segment 3. Pilot must keep a minimal horizontal deflection when intercepting CVA 022 Radial (minimum limit 0.1)
FLYING_VOR_END_DME_CVA = 0.5 # End criteria for segment 4. Pilot must fly to CVA Vortac (End when minimum distance 0.5 nm)
HEADING_036_RADIAL = 36 # End criteria for section A (Segment 5). Heading is 036°

# --- Modified Segment Detection Logic (State Machine Approach) ---
def detect_missed_approach_segment(df):
    df = df.copy()
    df["segment"] = "Undetermined"

    current_segment = 0 # Initially, not in a segment

    for i, row in df.iterrows():
        if current_segment == 0:
            # 0. No segments yet
            if row[NAV1_DME_COL] <= FINAL_APP_START_DME: # 6.9
                # Pilot finally reaches 6.9 DME from runway, transition to segment 1 (Begin task)
                df.at[i, "segment"] = "1_Final_Approach"
                current_segment = 1

        elif current_segment == 1:
            # 1. Final Approach
            df.at[i, "segment"] = "1_Final_Approach"
            if row[ALT_MSL_COL] < FINAL_APP_END_ALT_MSL:
                # As long as pilot is at least 900 ft. MSL, remain on segment 1
                # The moment pilot drops below 900 ft., transition to segment 2
                df.at[i, "segment"] = "2_Transition_Missed"
                current_segment = 2

        elif current_segment == 2:
            # 2. Transition to Missed
            df.at[i, "segment"] = "2_Transition_Missed"
            if round(row[ALT_MSL_COL]) >= TRANSITION_END_ALT_MSL:
                # As long as pilot is below 1200 ft. MSL, remain on segment 2
                # When pilot reached at least 1200 ft. MSL, transition to segment 3
                df.at[i, "segment"] = "3a_Climb_to_Intercept"
                current_segment = 3

        elif current_segment == 3:
            # 3a. Climb to Intercept
            df.at[i, "segment"] = "3a_Climb_to_Intercept"
            if (round(row[ALT_MSL_COL]) >= CLIMB_INTERCEPT_END_ALT_MSL):
                # As long as pilot is below 3100 ft. MSL, they are still in segment 3a (first part of segment 3).
                # If pilot ascended to at least 3100 ft. MSL, they will move to segment 3b (second part of segment 3)
                df.at[i, "segment"] = "3b_Climb_to_Intercept"
                current_segment = 3.5

        elif current_segment == 3.5:
            # 3b. Climb to Intercept
            df.at[i, "segment"] = "3b_Climb_to_Intercept"
            if (abs(row[NAV1_HDEF_COL]) <= NAV1_HORIZONTAL_DEF_MIN):
                # As long as pilot has not intercepted the CVA 022 Radial, they are in segment 3b (second part of segment 3).
                df.at[i, "segment"] = "4_Flying_to_VOR"
                current_segment = 4

        elif current_segment == 4:
            # Flying to VOR
            df.at[i, "segment"] = "4_Flying_to_VOR"
            if row[NAV1_FLAG_COL] == 0:
                # As long as pilot flag is not 0, remain on segment 4
                # When pilot is close enough to CVA, where flag is 0 (i.e. CVA is inteercepted), transition to segment 5
                df.at[i, "segment"] = "5_Holding" # Segment 5 (but not necessarily performing a section yet.. Pilot entered the cone of confusion)
                current_segment = 5

        # GOAL: Write code to indicate when pilot exits cone of confusion
        elif current_segment == 5:
            # 5. Cone of confusion (Pilot is right over the VOR)
            df.at[i, "segment"] = "5_Holding"

    df = df[df['segment'] != "Undetermined"] # Exclude parts that are undetermined
    df = df.reset_index(drop=True)

    df.to_csv('segments1-5.csv')

    return df

# During the missed approach, the pilot approaches the cone of confusion (i.e. when flying direclty above CVA VORTAC)
# The flag will become 0 in the cone of confusion, but will sometimes flucuate between 1 and 2
def detect_cone_of_confusion(df):
    df = df.copy()
    holding = df[df["segment"] == "5_Holding"]

    # Track the longest start and end sequence of consecutive 1 (to) and 2 (from) flags
    max_len = 0
    max_start = None
    max_end = None

    start = None
    end = None

    for i, row in holding.iterrows():
        if row[NAV1_FLAG_COL] in [1, 2]:
            if start is None:
                start = i
        elif row[NAV1_FLAG_COL] == 0: # End sequence of consecutive 1s and 2s
            if start is not None:
                end = i - 1
                length = end - start + 1
                if length > max_len:
                    max_len = length
                    max_start = start
                    max_end = end
                start = None

    # Catch if the sequence ends at the last index
    # (i.e. in case if pilot never enters CVA VORTAC for the second time after the lap)
    if start is not None:
        end = holding.index[-1]
        length = end - start + 1
        if length > max_len:
            max_len = length
            max_start = start
            max_end = end

    # # Edge case: If the longest consecutive sequence of 1s and 2s streches til the end of the dataframe
    # # (i.e. end has not been assigned yet)
    # if start > end:
    #     print('CASE DETECTED')
    #     max_end = len(df) - 1

    # Update original dataframes
    # (Any rows outisde of max_start and max_end indices get assigned as cone of confusion
    for i, row in holding.iterrows():
        if i < max_start or i > max_end:
            df.at[i, "segment"] = "5_COC"

    return df

# Get the rows that are currently 5_Holding and break them down into sections A, B, C, D
def detect_holding_sections(df):
    df = df.copy()
    holding = df[df["segment"] == "5_Holding"]

    current_section = "a"
    # elapsed_seconds_section_b = 0
    # section_b_first_row = True

    for i, row in holding.iterrows():
        if current_section == "a":
            df.at[i, "segment"] = "5_A_Holding"
            if (abs(row[HEADING_COL] - HEADING_036_RADIAL) <= 1) and (row[NAV1_FLAG_COL] == 1): # 036 radial and flag 1
                # Section A ends when pilot heading is around 36 degrees (Can tolerate slight deflections),
                # and is abeam the VOR (Switching flag from 2 -> 1)
                df.at[i, "segment"] = "5_B_Holding"
                current_section = "b"

        elif current_section == "b":
            df.at[i, "segment"] = "5_B_Holding"

            # Calculating time elapsed since 5_B_Holding
            # if section_b_first_row:
            #     section_b_first_row = False
            # else:
            #     elapsed_seconds_section_b += holding.at[i, TIME_COL] - holding.at[i - 1, TIME_COL] # Current time minus previous time

            if row[HEADING_COL] >= HEADING_036_RADIAL + 30: # Pilot must fly around 1 minute (60 seconds) and Right hand turn to intercept
                df.at[i, "segment"] = "5_C_Holding"
                current_section = "c"

        elif current_section == "c":
            df.at[i, "segment"] = "5_C_Holding"
            if abs(row[NAV1_HDEF_COL]) <= NAV1_HORIZONTAL_DEF_MIN: # Ends when aircraft intercepts the 036° radial inbound
                df.at[i, "segment"] = "5_D_Holding"
                current_section = "d"

        elif current_section == "d":
            # The rest is just section D, until reaching the next cone of confusion
            df.at[i, "segment"] = "5_D_Holding"

    return df

In [104]:
HEADING_COL

'hding,__mag'

## Task Scoring

Flying to VOR

In [97]:
def tolerance_score(value, target, tolerance): # Should only range between 0 and 1
    deviation = abs(value - target)
    score = 1 - (deviation / tolerance)
    score = 0 if score <= 0 else score
    return score

# ----- Constants to score VOR tracking -----
VOR_ALTITUDE_MAINTAIN = 6000
VOR_SPEED_MAINTAIN = 90
VOR_DEFLECTION = 0

def score_VOR(df):
    df = df.copy()
    df['score'] = 0.0 # Create new column

    for i, row in df.iterrows():
        total = 0
        if row['segment'] == 'VOR_Tracking':
            total += tolerance_score(row[ALT_MSL_COL], VOR_ALTITUDE_MAINTAIN, 100) # Altitude 6000 +- 100 ft.
            total += tolerance_score(row[IAS_COL], VOR_SPEED_MAINTAIN, 10) # Speed 90 +- 10 kts.
            total += tolerance_score(row[NAV1_HDEF_COL], VOR_DEFLECTION, 1) # Radial deflection within 1 dot on HSI
            df.at[i, 'score'] = total / 3 # Average

    return df

Missed Approach

In [98]:
# ----- Constants to score Missed Approach -----
SEG_1_H_SPEED_MAINTAIN = 90
SEG_1_V_SPEED_MAINTAIN = -1000
SEG_2_HEADING = 30
SEG_3A_H_SPEED_MAINTAIN = 74
SEG_3B_H_SPEED_MAINTAIN = 90
SEG_3B_ALT_MAINTAIN = 3100
SEG_4_ALTITUDE_MAINTAIN = 3100
SEG_4_HEADING = 202
SEG_4_H_SPEED_MAINTAIN = 90
SEG_5_ALTITUDE_MAINTAIN = 3100
SEG_5_H_SPEED_MAINTAIN = 90
SEG_5_A_TURNRATE = 23.5
SEG_5_B_HEADING = 36
SEG_5_C_TIME_SEC = 60
SEG_5_D_HEADING = 216

def score_missed_approach(df):
    df = df.copy()
    df['score'] = 0.0 # Create new column

    for i, row in df.iterrows():
        if row['segment'] == '1_Final_Approach':
            total = 0.0
            total += tolerance_score(row[NAV1_HDEF_COL], 0, 1) # Localizer Deflection within one dot
            total += tolerance_score(row[NAV1_VDEF_COL], 0, 0.5) # Glideslope Deflection within one-half scale
            total += tolerance_score(row[IAS_COL], SEG_1_H_SPEED_MAINTAIN, 10) # Airspeed 90 Knots +- 10
            total += 1.0 if row[VVI_COL] >= SEG_1_V_SPEED_MAINTAIN else 0.0 # Vertical speed >= -1000
            df.at[i, 'score'] = total / 4

        elif row['segment'] == '2_Transition_Missed':
            total = 0.0
            total += tolerance_score(row[HEADING_COL], SEG_2_HEADING, 5) # Heading 030° +- 5°
            total += 1.0 if row[VVI_COL] > 0 else 0.0 # Vertical Speed > 0 fpm
            df.at[i, 'score'] = total / 2

        elif row['segment'] == '3a_Climb_to_Intercept':
            total = 0.0
            total += tolerance_score(row[IAS_COL], SEG_3A_H_SPEED_MAINTAIN, 20) # Airspeed 74kts +- 20
            total += 1.0 if row[VVI_COL] > 0 else 0.0 # Vertical Speed >0 fpm
            df.at[i, 'score'] = total / 2

        elif row['segment'] == '3b_Climb_to_Intercept':
            total = 0.0
            total += tolerance_score(row[IAS_COL], SEG_3B_H_SPEED_MAINTAIN, 10) # Airspeed 90kts +- 10
            total += tolerance_score(row[ALT_MSL_COL], SEG_3B_ALT_MAINTAIN, 100) # Altitude 3100 ft MSL +- 100
            df.at[i, 'score'] = total / 2

        elif row['segment'] == '4_Flying_to_VOR':
            total = 0.0
            total += tolerance_score(row[ALT_MSL_COL], SEG_4_ALTITUDE_MAINTAIN, 100) # Altitude 3,100ft +- 100
            # total += tolerance_score(row[HEADING_COL], SEG_4_HEADING, 10) # Heading 202° +- 5°
            total += tolerance_score(row[IAS_COL], SEG_4_H_SPEED_MAINTAIN, 10) # Airspeed 90 Kts +- 10
            total += tolerance_score(row[NAV1_HDEF_COL], 0, 1) # Localizer Deflection within one dot
            df.at[i, 'score'] = total / 3

        elif row['segment'] == '5_COC':
            total = 0.0
            total += tolerance_score(row[ALT_MSL_COL], SEG_5_ALTITUDE_MAINTAIN, 100) # Altitude 3,100ft +- 100
            total += tolerance_score(row[IAS_COL], SEG_5_H_SPEED_MAINTAIN, 10) # Airspeed 90 Kts +- 10
            df.at[i, 'score'] = total / 2

        elif row['segment'] == '5_A_Holding':
            total = 0.0
            total += tolerance_score(row[ALT_MSL_COL], SEG_5_ALTITUDE_MAINTAIN, 100) # Altitude 3,100ft +- 100
            total += tolerance_score(row[IAS_COL], SEG_5_H_SPEED_MAINTAIN, 10) # Airspeed 90 Kts +- 10
            # total += tolerance_score(row[TURN_RATE_COL], SEG_5_A_TURNRATE, 10) # Rate of turn 23.5° (TODO: Raphael is to find out what units is this ref using?)
            df.at[i, 'score'] = total / 2

        elif row['segment'] == '5_B_Holding':
            total = 0.0
            total += tolerance_score(row[ALT_MSL_COL], SEG_5_ALTITUDE_MAINTAIN, 100) # Altitude 3,100ft +- 100
            total += tolerance_score(row[IAS_COL], SEG_5_H_SPEED_MAINTAIN, 10) # Airspeed 90 Kts +- 10
            total += tolerance_score(row[HEADING_COL], SEG_5_B_HEADING, 10) # Heading 036° +- 10
            df.at[i, 'score'] = total / 3

        elif row['segment'] == '5_C_Holding':
            total = 0.0
            total += tolerance_score(row[ALT_MSL_COL], SEG_5_ALTITUDE_MAINTAIN, 100) # Altitude 3,100ft +- 100
            total += tolerance_score(row[IAS_COL], SEG_5_H_SPEED_MAINTAIN, 10) # Airspeed 90 Kts +- 10
            # total += tolerance_score(row[TURN_RATE_COL], SEG_5_A_TURNRATE, 10) # Rate of turn 23.5° (TODO: Raphael is to find out what units is this ref using?)
            df.at[i, 'score'] = total / 2

        elif row['segment'] == '5_D_Holding':
            total = 0.0
            total += tolerance_score(row[ALT_MSL_COL], SEG_5_ALTITUDE_MAINTAIN, 100) # Altitude 3,100ft +- 100
            total += tolerance_score(row[IAS_COL], SEG_5_H_SPEED_MAINTAIN, 10) # Airspeed 90 Kts +- 10
            total += tolerance_score(row[HEADING_COL], SEG_5_D_HEADING, 10) # Heading 216° +- 10
            df.at[i, 'score'] = total / 3

    return df

## Data cleaning/processing

In [99]:
def filter_paused(df):
    df = df.copy()
    df = df[df[IS_PAUSED] == 0]
    df = df.reset_index(drop=True)
    return df

def check_data(df, task_type):
    df = df.copy()
    print(f"Read {len(df)} rows...")
    if df.empty:
        print("File is empty."); return None # REMOVED: No row_scores_df to return

    print("Checking required columns...")

    # df = fill_trailing_zeroes(df)

    # Validate if columns exist
    all_cols_ok = True

    if task_type == "missed_approach":
        cols_to_check = MISSED_APPROACH_COLS
    elif task_type == "vor_tracking":
        cols_to_check = VOR_TRACKING_COLS

    for col in cols_to_check:
        if col not in df.columns:
            print(f"  ERROR: Required column '{col}' not found for {task_type} task.")
            all_cols_ok = False

    if not all_cols_ok:
        print("Data checking failed. Exiting.")
        return None

    # # Data trimming (Trim off irrelevant parts of X-Plane that was paused)
    # df = df[df['is_paused'] == 0]

    print("DATA CHECKING SUCCESSFUL.")
    return df

# --- Main Execution Logic ---
def process_flight_data(filename, task_type="missed_approach"):
    # Processes a single flight data CSV file.

    print(f"\n--- Processing {filename} (Task: {task_type}) ---")

    # ---- Data Cleaning ----
    df = pd.read_csv(filename)
    df = check_data(df, task_type)

    if df is None:
        return 0

    # ---- Detecting segments and scoring them ----
    print("Detecting segments...")
    if task_type == "missed_approach":
        df = detect_missed_approach_segment(df) # Get segments 1-5
        df = detect_cone_of_confusion(df) # Get cone of confusion within segment 5
        df = detect_holding_sections(df) # Get sections A-B within segment 5
        df = filter_paused(df)
        df = score_missed_approach(df)
    elif task_type == "vor_tracking":
        df = detect_vor_tracking_segment(df)
        df = filter_paused(df)
        df = score_VOR(df)
    else:
        print(f"Unknown task type: {task_type}")
        return None # REMOVED: No row_scores_df to return

    segment_counts = df['segment'].value_counts()
    if not segment_counts.empty:
        print("Segment Detection Counts:\n", segment_counts)
    else:
        print("No segments detected.")

    return df

In [100]:
def analyze_score(df, filename, task_type='missed_approach'):
    print(f"--- Aanalyzing scores for {task_type} ---")

    df = df.copy()
    filename = filepath.split('/')[-1].split('.')[0]

    if task_type == "missed_approach":
        df_seg_5 = df[df["segment"].str.contains("5_")]
        # Create the DataFrame with row indices "time" and "avg_score"
        cols = ["1_Final_Approach", "2_Transition_Missed", "3a_Climb_to_Intercept", "3b_Climb_to_Intercept", "4_Flying_to_VOR",
                "5_COC", "5_A_Holding", "5_B_Holding", "5_C_Holding", "5_D_Holding", "5_OVERALL", "OVERALL"]
        result = pd.DataFrame(np.nan, index=["time (seconds)", "avg_score"], columns=cols)

        # Score
        segment_scores = df.groupby("segment")["score"].mean()
        for segment, score in segment_scores.items():
            result.at["avg_score", segment] = score
        result.at["avg_score", "OVERALL"] = df["score"].mean()

        if np.isnan(df_seg_5["score"].mean()): # Segment 5 DNE
            pass
        else:
            result.at["avg_score", "5_OVERALL"] = df_seg_5["score"].mean()

        # Time
        segment_times = df.groupby("segment")["sys_unix_time"].agg(lambda x: (x.iloc[-1] - x.iloc[0]) / 1000)
        for segment, time in segment_times.items():
            result.at["time (seconds)", segment] = time
        result.at["time (seconds)", "OVERALL"] = (df["sys_unix_time"].iloc[-1] - df["sys_unix_time"].iloc[0]) / 1000

        if np.isnan(df_seg_5["score"].mean()): # Segment 5 DNE
            pass
        else:
            result.at["time (seconds)", "5_OVERALL"] = (df_seg_5["sys_unix_time"].iloc[-1] - df_seg_5["sys_unix_time"].iloc[0]) / 1000

        # If not all segments are available, then automatic fail
        if result.isna().any().any():
            result.at["avg_score", "OVERALL"] = 0

    # Getting scores
    elif task_type == "vor_tracking":
        # Create the DataFrame with row indices "time" and "avg_score"
        result = pd.DataFrame(np.nan, index=["time (seconds)", "avg_score"], columns=["VOR_Tracking_OVERALL"])
        result.at["avg_score", "VOR_Tracking_OVERALL"] = df['score'].mean()
        result.at["time (seconds)", "VOR_Tracking_OVERALL"] = (df["sys_unix_time"].iloc[-1] - df["sys_unix_time"].iloc[0]) / 1000

    result.to_csv(os.path.join(SUMMARY_SCORE_OUTPUTS_DIR, f'{filename}_summary.csv'), index=True)
    print(f"Analysis successful. Saved to {os.path.join(SUMMARY_SCORE_OUTPUTS_DIR, f'{filename}_summary.csv')}\n")

    return result

# Main Block

Certain data files require manual modification by the researcher. All original .csv files are stored in `/content/drive/MyDrive/Missed Approach/Datarefs/Original Data`. Some files in `/content/drive/MyDrive/Missed Approach Scoring/Datarefs` may be manually modified.

 Data files modified:
 - `P1_T2`: Manually trimmed as participant flew 2 laps on the hold
 - `P10_T1`: Flew far past the VOR without stop. Trimmed portion past the VOR
 - `P12_T2`, `P4_T2`, `P17_T2`, `P20_T2`, `P21_T2`: Participant(s) did not drop enough below the decision altitude during final approach, but flew well overall. The lowest altitude was set to 899 to get all segments.
 - `P23_T2`: At the start of section A of the hold, the pilot did not switch his course indicator soon enough, thus causing the 'pi1N1,flag' values to entirely show 2 (from indicator) during 5A. We manually changed some values in the 'pi1N1,flag' column during 5A to have a "to" indicator (value of 1).

 Other notes:
 - Unable to run analysis for `P6_T2` due to TypeError: '<' not supported between instances of 'int' and 'NoneType.' But this pilot has faield according to participant notes

In [116]:
for p in [1, 2, 4, 5, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]:
    filepath = f"/content/drive/MyDrive/Missed Approach Scoring/Datarefs/P{p}_T2.csv"
    filename = filepath.split('/')[-1].split('.')[0]
    task = 'missed_approach' if 'T2' in filepath else 'vor_tracking' # missed_approach or vor_tracking

    if not os.path.exists(filepath): # Creating dummy data for demonstration
        print(f"Warning: File {filepath} not found.")
    else:
        print(f"File {filepath} exists. Generating scores...")

        # --- Computing Scores ---
        pilot_result = process_flight_data(filepath, task_type=task)

        pilot_result.to_csv(os.path.join(SCORE_OUTPUTS_DIR, f"{filename}_output.csv"), index=False)
        print(f"Scores saved to {os.path.join(SCORE_OUTPUTS_DIR, f'{filename}_output.csv')}\n")

        analyze_score(pilot_result, filename, task)

        print("--- Script Finished ---")

File /content/drive/MyDrive/Missed Approach Scoring/Datarefs/P1_T2.csv exists. Generating scores...

--- Processing /content/drive/MyDrive/Missed Approach Scoring/Datarefs/P1_T2.csv (Task: missed_approach) ---
Read 3611 rows...
Checking required columns...
DATA CHECKING SUCCESSFUL.
Detecting segments...
MAX START: 2039
MAX END: 2578
Segment Detection Counts:
 segment
1_Final_Approach         594
3a_Climb_to_Intercept    550
3b_Climb_to_Intercept    454
4_Flying_to_VOR          344
5_A_Holding              158
5_B_Holding              153
5_C_Holding              132
5_D_Holding               97
5_COC                     69
2_Transition_Missed       66
Name: count, dtype: int64
Scores saved to /content/drive/MyDrive/Missed Approach Scoring/Score Outputs/P1_T2_output.csv

--- Aanalyzing scores for missed_approach ---
Analysis successful. Saved to /content/drive/MyDrive/Missed Approach Scoring/Score Outputs/Summary/P1_T2_summary.csv

--- Script Finished ---
File /content/drive/MyDrive/Mis

Data Trimming (If Needed)

In [87]:
# df = pd.read_csv('/content/drive/MyDrive/Missed Approach Scoring/Datarefs/Original Data/Original Copy of P10_T1.csv')

# df = df.iloc[:2485]

# df.to_csv('/content/drive/MyDrive/Missed Approach Scoring/Datarefs/P10_T1.csv', index=False)

Manually computing the time on task, if certain data files get an error

In [88]:
# sample = pd.read_csv('/content/drive/MyDrive/Missed Approach Scoring/sample.csv')
# (sample.iloc[-1]['sys_unix_time'] - sample.iloc[0]['sys_unix_time']) / 1000

Manually setting the lowest altitude to 899 (In case if pilots did not dip below exactly 900 ft. MSL)

In [89]:
# filename = 'P21_T2'
# df = pd.read_csv(f'/content/drive/MyDrive/Missed Approach Scoring/Datarefs/{filename}.csv')

# min = df['p-alt,ftMSL'].min()

# df['p-alt,ftMSL'] = df['p-alt,ftMSL'].replace(min, 899)

# df['p-alt,ftMSL'].min()

# df.to_csv(f'/content/drive/MyDrive/Missed Approach Scoring/Datarefs/{filename}.csv', index=False)

In [90]:
# tasks = os.listdir('/content/drive/MyDrive/Missed Approach Scoring/Score Outputs')
# tasks = [t for t in tasks if 'P' in t and 'T2' in t]


# for t in tasks:
#     df = pd.read_csv(f'/content/drive/MyDrive/Missed Approach Scoring/Score Outputs/{t}')

#     if 1 in df[IS_PAUSED].to_numpy():
#         print(t)

In [115]:
# df = pd.read_csv('/content/drive/MyDrive/Missed Approach Scoring/Datarefs/P23_T2.csv')

# # df.iloc[3430] # LAST ROW
# # df.iloc[3430 - 35] # FIRST ROW

# for i, row in df.iloc[3430 - 35:3430 + 1].iterrows():
#     df.at[i, 'pi1N1,flag'] = 1


# df.to_csv('/content/drive/MyDrive/Missed Approach Scoring/Datarefs/P23_T2.csv', index=False)